In [1]:
%run Futures.ipynb
%run Options.ipynb

In [2]:
import pandas as pd
import numpy as np
from datetime import date, time, datetime
from py_vollib.black_scholes_merton.implied_volatility import implied_volatility as iv
from math import exp, sqrt, log, fabs
import matplotlib.pyplot as plt
from itertools import repeat
import os
import calendar 
import glob

In [3]:
work_dir = os.getcwd()
work_dir
Option_path = os.path.join(work_dir, 'Data', 'Option')
Future_path = os.path.join(work_dir, 'Data', 'Futures')
OptionOpen_path = os.path.join(work_dir, 'Data', 'OptionOpen')
OptionClose_path = os.path.join(work_dir, 'Data', 'OptionClose')
Rate_path = os.path.join(work_dir, 'Data', 'Rate')

In [4]:
Option_path

'C:\\Users\\Evan\\Documents\\GitHub\\TXO_TX_Strategy\\Data\\Option'

## Calculate the Date Difference

In [5]:
def dates_delta(start_time, maturity):
    '''
        Compute the distance between two TRADE_DATEs
        Already consider about the weekly options
    '''
    
    s = start_time
    
    d0 = date(year = int(s/10000), month = int((s % 10000)/100), day = int(s % 100))
    if maturity[-2] == 'W':
        m = maturity[:5]
        year = int(int(maturity[:6])/100)
        month = int(int(maturity[:6]) % 100)
        weekidx = int(maturity[-1]) - 1
        d1 = get_expireDate(year, month, week = weekidx)
        
    else: 
        year = int(int(maturity[:6])/100)
        month = int(int(maturity[:6]) % 100)
        d1 = get_expireDate(year, month)

    t = np.busday_count(d0, d1)/252
    if t < 0:
        print("The input dates error!")
    return t

In [6]:
def trans_dt(d):
    d_new = date(year = int(d/10000), month = int((d % 10000)/100), day = int(d % 100))
    return d_new

In [7]:
def get_expireDate(year, month, week = 2):
    c = calendar.Calendar(firstweekday=calendar.SATURDAY)
    monthcal = c.monthdatescalendar(year, month)
    monthly_expire_date = monthcal[week][-3]
    return monthly_expire_date

In [8]:
def get_Future_List(): 
    extension = 'csv'
    os.chdir(Future_path)
    Fut_csv_List = glob.glob('*.{}'.format(extension))
    return Fut_csv_List

In [9]:
def get_Option_List(): 
    extension = 'csv'
    os.chdir(Option_path)
    Opt_csv_List = glob.glob('*.{}'.format(extension))
    return Opt_csv_List

In [10]:
def get_zcb_rate(transDate):
    if ~isinstance(transDate, date): 
        transDate = np.datetime64(trans_dt(transDate))
    
    rate =  pd.read_csv(os.path.join(Rate_path, 'ZCB.csv'))
    rate['Date'] = pd.to_datetime(rate['Date'])
    zcb = float(rate[rate.Date == transDate]['ZCB'])
    
    return zcb

In [11]:
def MK_disc(S, K, t): 
    'Maturity and Moneyness Discount'
    t = t * 252 # transform into days
    try:
        
        m = float(K/S - 1) # moneyness
        M = max(1, t/30.0) # days to month; at least one-month
        w = exp(-(m**2)/2 - (M - 1)**2)
    except ZeroDivisionError:
        M = max(1, t/30.0)
        w = exp(- (M - 1)**2)
    return w

In [12]:
def call_put_pair(Opt, Fut):
    '''
        Pair the Put Call Options
    '''
    pair_list = []
    pair_dic = {}
    
    length = Opt.get_length()
    
    for i in range(length):
        
        transDate= Opt.get_transDate()
        maturity = Opt.get_maturity(i)
        t = dates_delta(transDate , maturity) 
        K = Opt.get_excercisePrice(i)
        right = Opt.get_right(i)
        interval = Opt.get_interval()
        
        dict_key = "K:" + str(K) + "/" + "T:" + str(td)
        
        S = float(Fut.get_TransPrice(transTime))
        price = Opt.get_transPrice(i)
        
        cur_vol = Opt.get_volumn(i) * MK_disc(S, K, t) # current vol

#=============================================================================
#   Transform the Dict into Pair            
#=============================================================================

        if dict_key not in pair_dic:

            pair_dic[dict_key] = len(pair_dic)
            if right == "C":
                pair_list.append([Opt.df.iloc[i], [], cur_vol])
            if right == "P":
                pair_list.append([[], Opt.df.iloc[i], cur_vol])
                
        else:
            if right == "C":
                if len(pair_list[pair_dic[dict_key]][0]) < 1:
                    pair_list[pair_dic[dict_key]][0] = Opt.df.iloc[i]
                    pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol
                    
                if interval == "Openinterval": 
                    if pair_list[pair_dic[dict_key]][0]['TransTime'] > Opt.df['TransTime'][i]:
                        pair_list[pair_dic[dict_key]][0] = Opt.df.iloc[i]
                        pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol
                    else:
                        pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol
                        
                elif interval == "Closeinterval": 
                    if pair_list[pair_dic[dict_key]][0]['TransTime'] < Opt.df['TransTime'][i]:
                        pair_list[pair_dic[dict_key]][0] = Opt.df.iloc[i]
                        pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol
                    else:
                        pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol
                
            if right == "P":
                if len(pair_list[pair_dic[dict_key]][1]) < 1:
                    pair_list[pair_dic[dict_key]][1] = df.iloc[i]
                    pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol
                    
                if interval == "Openinterval": 
                    if pair_list[pair_dic[dict_key]][0]['TransTime'] > Opt.df['TransTime'][i]:
                        pair_list[pair_dic[dict_key]][0] = Opt.df.iloc[i]
                        pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol
                    else:
                        pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol
                        
                elif interval == "Closeinterval": 
                    if pair_list[pair_dic[dict_key]][0]['TransTime'] < Opt.df['TransTime'][i]:
                        pair_list[pair_dic[dict_key]][0] = Opt.df.iloc[i]
                        pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol
                    else:
                        pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol
    
    return pair_list

In [12]:
if __name__ ==  '__main__':
    Opt_csv_List = get_Option_List()
    dataCsv = Opt_csv_List[0]
    path = OptionOpen_path
    Opt = Options(path, dataCsv)
    Fut_csv_list = get_Future_List()
    FutCsv = Fut_csv_list[0]
    Fut = Futures(FutCsv)

In [18]:
i = 0
transDate = Opt.get_transDate()
maturity = Opt.get_maturity(i)
t = dates_delta(transDate , maturity) 
K = Opt.get_excercisePrice(i)
right = Opt.get_right(i)

dict_key = "K:" + str(K) + "/" + "T:" +str(t)

transTime = Opt.get_transTime(i)
S = float(Fut.get_TransPrice(transTime))
price = Opt.get_transPrice(i)

vol = Opt.get_volumn(i) * MK_disc(S, K, t) 

In [19]:
cur_vol = vol

In [20]:
pair_list = []
pair_dic = {}
if dict_key not in pair_dic:

    pair_dic[dict_key] = len(pair_dic)
    if right == "C":
        pair_list.append([Opt.df.iloc[i], [], cur_vol])
    if right == "P":
        pair_list.append([[], Opt.df.iloc[i], cur_vol])

else:
    if right == "C":
        if len(pair_list[pair_dic[dict_key]][0]) < 1:
            pair_list[pair_dic[dict_key]][0] = Opt.df.iloc[i]
            pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol

        if interval == "Openinterval": 
            if pair_list[pair_dic[dict_key]][0]['TransTime'] > Opt.df['TransTime'][i]:
                pair_list[pair_dic[dict_key]][0] = Opt.df.iloc[i]
                pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol
            else:
                pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol

        elif interval == "Closeinterval": 
            if pair_list[pair_dic[dict_key]][0]['TransTime'] < Opt.df['TransTime'][i]:
                pair_list[pair_dic[dict_key]][0] = Opt.df.iloc[i]
                pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol
            else:
                pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol

    if right == "P":
        if len(pair_list[pair_dic[dict_key]][1]) < 1:
            pair_list[pair_dic[dict_key]][1] = df.iloc[i]
            pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol

        if interval == "Openinterval": 
            if pair_list[pair_dic[dict_key]][0]['TransTime'] > Opt.df['TransTime'][i]:
                pair_list[pair_dic[dict_key]][0] = Opt.df.iloc[i]
                pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol
            else:
                pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol

        elif interval == "Closeinterval": 
            if pair_list[pair_dic[dict_key]][0]['TransTime'] < Opt.df['TransTime'][i]:
                pair_list[pair_dic[dict_key]][0] = Opt.df.iloc[i]
                pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol
            else:
                pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol

In [22]:
pair_dic

{'K:8200/T:0.011904761904761904': 0}